In [1]:
import numpy as np
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import random
import string


In [2]:
rng = np.random.default_rng(42)
random.seed(42)

SEASON = "2024-25"
TEAM = "LAKERS"
VENUE = "Crypto.com Arena"

# A small opponent pool (edit as you like)
OPPONENTS = [
    "Warriors","Celtics","Suns","Clippers","Nuggets","Mavericks","Kings","Spurs",
    "Bulls","Heat","Knicks","Nets","76ers","Bucks","Raptors","Pelicans"
]

TICKET_CHANNELS = ["primary_web", "primary_app", "box_office", "secondary_market", "corporate"]
TICKET_TIERS = ["upper", "mid", "lower", "courtside", "suite"]
MERCH_CATEGORIES = ["jerseys", "tshirts", "hoodies", "hats", "collectibles", "kids", "other"]
PROMO_TYPES = ["none", "bobblehead", "theme_night", "discount_bundle", "giveaway", "partner_activation"]

def random_game_dates(start_date="2024-10-20", n=41):
    # 41 home games; synthetic dates spread across ~6 months
    start = pd.Timestamp(start_date)
    dates = []
    d = start
    while len(dates) < n:
        # 2–4 days between home games
        d += pd.Timedelta(days=int(rng.integers(2, 5)))
        # keep within season range
        if d.month <= 4 or d.year == start.year:
            dates.append(d)
    return dates[:n]

def demand_index(opponent, day_of_week, promo_type, is_weekend):
    # crude demand scoring (0.6–1.6)
    base = 1.0
    marquee = {"Warriors","Celtics","Clippers","Nuggets","Suns","Knicks"}
    if opponent in marquee:
        base += 0.25
    if is_weekend:
        base += 0.10
    if promo_type in {"bobblehead","theme_night","partner_activation"}:
        base += 0.10
    if day_of_week in {"Mon","Tue"}:
        base -= 0.07
    return float(np.clip(base + rng.normal(0, 0.05), 0.6, 1.6))

def capacity_by_mix():
    # Total capacity with suites etc. (rough)
    # You can replace with exact figures if you have them.
    return int(rng.integers(17500, 19600))

def tier_share(tier):
    # approximate seat mix
    shares = {"upper":0.45, "mid":0.25, "lower":0.22, "courtside":0.02, "suite":0.06}
    return shares[tier]

def base_price(tier):
    # baseline ticket price by tier (USD)
    return {"upper":55, "mid":120, "lower":210, "courtside":1800, "suite":750}[tier]

def channel_multiplier(channel):
    return {"primary_web":1.00, "primary_app":0.98, "box_office":1.02, "secondary_market":1.12, "corporate":1.08}[channel]

def merch_spend_per_cap(demand, promo_type):
    # per-attendee merch spend
    base = 11.5
    if promo_type in {"theme_night","bobblehead","partner_activation"}:
        base += 1.2
    return float(np.clip(base * demand + rng.normal(0, 1.0), 6.0, 30.0))

def concession_spend_per_cap(demand, is_weekend):
    base = 18.0 + (2.0 if is_weekend else 0.0)
    return float(np.clip(base * (0.95 + 0.12*(demand-1.0)) + rng.normal(0, 1.3), 10.0, 45.0))

dates = random_game_dates(n=41)

games = []
for i, d in enumerate(dates, start=1):
    opponent = random.choice(OPPONENTS)
    dow = d.strftime("%a")
    is_weekend = dow in {"Fri","Sat","Sun"}
    promo_type = random.choices(PROMO_TYPES, weights=[50,10,10,10,10,10], k=1)[0]
    cap = capacity_by_mix()
    demand = demand_index(opponent, dow, promo_type, is_weekend)
    # attendance rate influenced by demand
    attendance = int(np.clip(cap * (0.84 + 0.14*(demand-1.0)) + rng.normal(0, 350), cap*0.70, cap))
    
    games.append({
        "game_id": f"{SEASON.replace('-','')}_HOME_{i:02d}",
        "season": SEASON,
        "team": TEAM,
        "opponent": opponent,
        "date": d.date().isoformat(),
        "day_of_week": dow,
        "venue": VENUE,
        "promo_type": promo_type,
        "capacity": cap,
        "attendance": attendance,
        "demand_index": round(demand, 3),
        "is_weekend": int(is_weekend)
    })

df_games = pd.DataFrame(games)
df_games.head()


,game_id,season,team,opponent,date,day_of_week,venue,promo_type,capacity,attendance,demand_index,is_weekend
0,202425_HOME_01,2024-25,LAKERS,Clippers,2024-10-22,Tue,Crypto.com Arena,none,19091,16854,1.146,0
1,202425_HOME_02,2024-25,LAKERS,Bulls,2024-10-26,Sat,Crypto.com Arena,none,18435,15564,1.079,1
2,202425_HOME_03,2024-25,LAKERS,Nuggets,2024-10-29,Tue,Crypto.com Arena,discount_bundle,19375,16963,1.207,0
3,202425_HOME_04,2024-25,LAKERS,Suns,2024-11-01,Fri,Crypto.com Arena,bobblehead,18647,17644,1.472,1
4,202425_HOME_05,2024-25,LAKERS,Celtics,2024-11-04,Mon,Crypto.com Arena,none,17824,15191,1.160,0


In [3]:
ticket_rows = []

for _, g in df_games.iterrows():
    cap = g["capacity"]
    demand = g["demand_index"]
    attendance = g["attendance"]
    
    # allocate tickets across tiers
    tier_counts = {}
    for tier in TICKET_TIERS:
        tier_counts[tier] = int(round(attendance * tier_share(tier)))
    # fix rounding drift
    drift = attendance - sum(tier_counts.values())
    tier_counts["upper"] += drift
    
    for tier in TICKET_TIERS:
        tier_qty = tier_counts[tier]
        # allocate across channels (weights vary)
        if tier in {"courtside","suite"}:
            weights = np.array([0.10,0.08,0.02,0.05,0.75])  # corporate heavy
        else:
            weights = np.array([0.45,0.18,0.05,0.25,0.07])
        weights = weights / weights.sum()
        channel_alloc = rng.multinomial(tier_qty, weights)
        
        for channel, qty in zip(TICKET_CHANNELS, channel_alloc):
            if qty == 0:
                continue
            price = base_price(tier) * channel_multiplier(channel) * demand
            # add noise, keep positive
            avg_price = float(np.clip(price + rng.normal(0, price*0.06), 10, None))
            gross = avg_price * qty
            
            ticket_rows.append({
                "game_id": g["game_id"],
                "tier": tier,
                "channel": channel,
                "tickets_sold": int(qty),
                "avg_ticket_price_usd": round(avg_price, 2),
                "ticket_gross_usd": round(gross, 2)
            })

df_ticket_sales = pd.DataFrame(ticket_rows)
df_ticket_sales.head()


,game_id,tier,channel,tickets_sold,avg_ticket_price_usd,ticket_gross_usd
0,202425_HOME_01,upper,primary_web,3444,57.82,199144.64
1,202425_HOME_01,upper,primary_app,1361,64.12,87271.91
2,202425_HOME_01,upper,box_office,400,63.43,25373.36
3,202425_HOME_01,upper,secondary_market,1824,64.36,117399.62
4,202425_HOME_01,upper,corporate,555,63.92,35478.06


In [4]:
merch_rows = []

for _, g in df_games.iterrows():
    demand = g["demand_index"]
    attendance = g["attendance"]
    promo = g["promo_type"]
    
    per_cap = merch_spend_per_cap(demand, promo)
    total_merch = per_cap * attendance
    
    # category mix (edit freely)
    mix = np.array([0.22,0.28,0.16,0.12,0.08,0.06,0.08])
    mix = mix / mix.sum()
    cat_sales = total_merch * mix
    
    for cat, rev in zip(MERCH_CATEGORIES, cat_sales):
        # approximate AOV by category
        aov = {
            "jerseys": 135, "tshirts": 38, "hoodies": 78, "hats": 32,
            "collectibles": 25, "kids": 30, "other": 20
        }[cat]
        units = int(max(0, round(rev / aov + rng.normal(0, (rev / aov)*0.08))))
        merch_rows.append({
            "game_id": g["game_id"],
            "category": cat,
            "units_sold": units,
            "revenue_usd": round(float(units * aov), 2),
            "aov_usd": float(aov)
        })

df_merch_sales = pd.DataFrame(merch_rows)
df_merch_sales.head()


,game_id,category,units_sold,revenue_usd,aov_usd
0,202425_HOME_01,jerseys,422,56970.0,135.0
1,202425_HOME_01,tshirts,1709,64942.0,38.0
2,202425_HOME_01,hoodies,495,38610.0,78.0
3,202425_HOME_01,hats,738,23616.0,32.0
4,202425_HOME_01,collectibles,698,17450.0,25.0


In [5]:
other_rows = []

for _, g in df_games.iterrows():
    demand = g["demand_index"]
    attendance = g["attendance"]
    is_weekend = bool(g["is_weekend"])
    
    concessions = concession_spend_per_cap(demand, is_weekend) * attendance
    parking = float(np.clip((6.5 + 1.2*(demand-1.0)) * (attendance*0.35) + rng.normal(0, 800), 0, None))
    
    other_rows.append({
        "game_id": g["game_id"],
        "concessions_revenue_usd": round(float(concessions), 2),
        "parking_revenue_usd": round(parking, 2)
    })

df_other_sales = pd.DataFrame(other_rows)
df_other_sales.head()


,game_id,concessions_revenue_usd,parking_revenue_usd
0,202425_HOME_01,290981.20,40223.18
1,202425_HOME_02,272327.76,35759.36
2,202425_HOME_03,320249.63,39730.97
3,202425_HOME_04,342064.75,44568.64
4,202425_HOME_05,258969.14,36463.96


In [6]:
DB_PATH = "lakers_sales_kb.sqlite"

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# Core tables
cur.executescript("""
PRAGMA foreign_keys = ON;

DROP TABLE IF EXISTS games;
DROP TABLE IF EXISTS ticket_sales;
DROP TABLE IF EXISTS merch_sales;
DROP TABLE IF EXISTS other_sales;
DROP TABLE IF EXISTS kb_notes;
DROP TABLE IF EXISTS kb_note_tags;
DROP TABLE IF EXISTS kb_tags;

CREATE TABLE games (
  game_id TEXT PRIMARY KEY,
  season TEXT,
  team TEXT,
  opponent TEXT,
  date TEXT,
  day_of_week TEXT,
  venue TEXT,
  promo_type TEXT,
  capacity INTEGER,
  attendance INTEGER,
  demand_index REAL,
  is_weekend INTEGER
);

CREATE TABLE ticket_sales (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  game_id TEXT,
  tier TEXT,
  channel TEXT,
  tickets_sold INTEGER,
  avg_ticket_price_usd REAL,
  ticket_gross_usd REAL,
  FOREIGN KEY(game_id) REFERENCES games(game_id)
);

CREATE TABLE merch_sales (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  game_id TEXT,
  category TEXT,
  units_sold INTEGER,
  revenue_usd REAL,
  aov_usd REAL,
  FOREIGN KEY(game_id) REFERENCES games(game_id)
);

CREATE TABLE other_sales (
  game_id TEXT PRIMARY KEY,
  concessions_revenue_usd REAL,
  parking_revenue_usd REAL,
  FOREIGN KEY(game_id) REFERENCES games(game_id)
);

-- Collaborative KB tables
CREATE TABLE kb_notes (
  note_id INTEGER PRIMARY KEY AUTOINCREMENT,
  created_at TEXT NOT NULL,
  author TEXT NOT NULL,
  game_id TEXT,
  note_type TEXT NOT NULL,        -- e.g. "technique", "insight", "promo_postmortem", "ops"
  title TEXT NOT NULL,
  content TEXT NOT NULL,
  source TEXT,                    -- optional: link, report name, "internal", etc.
  FOREIGN KEY(game_id) REFERENCES games(game_id)
);

CREATE TABLE kb_tags (
  tag_id INTEGER PRIMARY KEY AUTOINCREMENT,
  tag TEXT UNIQUE NOT NULL
);

CREATE TABLE kb_note_tags (
  note_id INTEGER,
  tag_id INTEGER,
  PRIMARY KEY(note_id, tag_id),
  FOREIGN KEY(note_id) REFERENCES kb_notes(note_id),
  FOREIGN KEY(tag_id) REFERENCES kb_tags(tag_id)
);

-- Full-text search (FTS5) for notes
DROP TABLE IF EXISTS kb_notes_fts;
CREATE VIRTUAL TABLE kb_notes_fts USING fts5(
  title, content, author, note_type, game_id,
  content='kb_notes',
  content_rowid='note_id'
);

-- Keep FTS in sync
DROP TRIGGER IF EXISTS kb_notes_ai;
DROP TRIGGER IF EXISTS kb_notes_ad;
DROP TRIGGER IF EXISTS kb_notes_au;

CREATE TRIGGER kb_notes_ai AFTER INSERT ON kb_notes BEGIN
  INSERT INTO kb_notes_fts(rowid, title, content, author, note_type, game_id)
  VALUES (new.note_id, new.title, new.content, new.author, new.note_type, new.game_id);
END;

CREATE TRIGGER kb_notes_ad AFTER DELETE ON kb_notes BEGIN
  INSERT INTO kb_notes_fts(kb_notes_fts, rowid, title, content, author, note_type, game_id)
  VALUES('delete', old.note_id, old.title, old.content, old.author, old.note_type, old.game_id);
END;

CREATE TRIGGER kb_notes_au AFTER UPDATE ON kb_notes BEGIN
  INSERT INTO kb_notes_fts(kb_notes_fts, rowid, title, content, author, note_type, game_id)
  VALUES('delete', old.note_id, old.title, old.content, old.author, old.note_type, old.game_id);
  INSERT INTO kb_notes_fts(rowid, title, content, author, note_type, game_id)
  VALUES (new.note_id, new.title, new.content, new.author, new.note_type, new.game_id);
END;
""")

# Load dataframes into SQLite
df_games.to_sql("games", conn, if_exists="append", index=False)
df_ticket_sales.to_sql("ticket_sales", conn, if_exists="append", index=False)
df_merch_sales.to_sql("merch_sales", conn, if_exists="append", index=False)
df_other_sales.to_sql("other_sales", conn, if_exists="append", index=False)

conn.commit()
conn.close()

DB_PATH


'lakers_sales_kb.sqlite'

In [7]:
def _ensure_tag(conn, tag: str) -> int:
    cur = conn.cursor()
    cur.execute("INSERT OR IGNORE INTO kb_tags(tag) VALUES (?)", (tag,))
    cur.execute("SELECT tag_id FROM kb_tags WHERE tag = ?", (tag,))
    return cur.fetchone()[0]

def add_kb_note(author, note_type, title, content, game_id=None, tags=None, source=None, db_path=DB_PATH):
    tags = tags or []
    created_at = datetime.utcnow().isoformat(timespec="seconds") + "Z"
    with sqlite3.connect(db_path) as conn:
        cur = conn.cursor()
        cur.execute("""
            INSERT INTO kb_notes(created_at, author, game_id, note_type, title, content, source)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (created_at, author, game_id, note_type, title, content, source))
        note_id = cur.lastrowid
        
        for t in tags:
            tag_id = _ensure_tag(conn, t.strip().lower())
            cur.execute("INSERT OR IGNORE INTO kb_note_tags(note_id, tag_id) VALUES (?, ?)", (note_id, tag_id))
        conn.commit()
    return note_id

def search_kb(query, limit=10, db_path=DB_PATH):
    with sqlite3.connect(db_path) as conn:
        q = """
        SELECT n.note_id, n.created_at, n.author, n.note_type, n.game_id, n.title
        FROM kb_notes_fts f
        JOIN kb_notes n ON n.note_id = f.rowid
        WHERE kb_notes_fts MATCH ?
        ORDER BY rank
        LIMIT ?;
        """
        return pd.read_sql_query(q, conn, params=(query, limit))

def get_note(note_id, db_path=DB_PATH):
    with sqlite3.connect(db_path) as conn:
        note = pd.read_sql_query("SELECT * FROM kb_notes WHERE note_id = ?", conn, params=(note_id,))
        tags = pd.read_sql_query("""
            SELECT t.tag
            FROM kb_note_tags nt
            JOIN kb_tags t ON t.tag_id = nt.tag_id
            WHERE nt.note_id = ?
            ORDER BY t.tag
        """, conn, params=(note_id,))
    return note, tags


In [8]:
# pick a couple games to attach notes to
some_game_ids = df_games["game_id"].sample(3, random_state=7).tolist()

add_kb_note(
    author="analyst_1",
    note_type="technique",
    title="Dynamic pricing bands by demand index",
    content=(
        "Use demand_index to define 4 pricing bands. Reprice upper/mid tiers weekly; "
        "hold lower/courtside changes to event-based triggers (marquee opponent, weekend, promo). "
        "Track elasticity by channel (primary vs secondary)."
    ),
    tags=["pricing", "tickets", "elasticity", "forecasting"]
)

add_kb_note(
    author="analyst_2",
    note_type="promo_postmortem",
    title="Theme night uplift on merch conversion",
    content=(
        "Theme nights tend to shift mix toward t-shirts/hats and increase per-cap spend. "
        "Measure uplift vs matched non-promo games (same weekday, similar opponent tier). "
        "Bundle limited merch SKUs with app push notifications 3–6 hours pre-tip."
    ),
    game_id=some_game_ids[0],
    tags=["merch", "promo", "conversion", "mobile"]
)

add_kb_note(
    author="ops_manager",
    note_type="insight",
    title="Secondary market spikes for weekend marquee games",
    content=(
        "Secondary market avg price increases disproportionately on Fri/Sat marquee opponents. "
        "Consider allocating more inventory to primary channels earlier and testing scarcity messaging."
    ),
    game_id=some_game_ids[1],
    tags=["secondary_market", "tickets", "weekend", "inventory"]
)

search_kb("merch OR pricing")


/var/folders/88/w4w1n8l12kd_z42_6mrnndmw0000gn/T/ipykernel_38706/2632202167.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  created_at = datetime.utcnow().isoformat(timespec="seconds") + "Z"


,note_id,created_at,author,note_type,game_id,title
0,1,2026-02-03T11:31:40Z,analyst_1,technique,None,Dynamic pricing bands by demand index
1,2,2026-02-03T11:31:40Z,analyst_2,promo_postmortem,202425_HOME_10,Theme night uplift on merch conversion


In [9]:
with sqlite3.connect(DB_PATH) as conn:
    # Total revenue per game (tickets + merch + other)
    q = """
    WITH t AS (
      SELECT game_id, SUM(ticket_gross_usd) AS ticket_rev
      FROM ticket_sales
      GROUP BY game_id
    ),
    m AS (
      SELECT game_id, SUM(revenue_usd) AS merch_rev
      FROM merch_sales
      GROUP BY game_id
    )
    SELECT
      g.game_id, g.date, g.opponent, g.promo_type, g.attendance, g.demand_index,
      COALESCE(t.ticket_rev,0) AS ticket_rev,
      COALESCE(m.merch_rev,0) AS merch_rev,
      COALESCE(o.concessions_revenue_usd,0) AS concessions_rev,
      COALESCE(o.parking_revenue_usd,0) AS parking_rev,
      (COALESCE(t.ticket_rev,0)+COALESCE(m.merch_rev,0)+COALESCE(o.concessions_revenue_usd,0)+COALESCE(o.parking_revenue_usd,0)) AS total_rev
    FROM games g
    LEFT JOIN t ON t.game_id = g.game_id
    LEFT JOIN m ON m.game_id = g.game_id
    LEFT JOIN other_sales o ON o.game_id = g.game_id
    ORDER BY total_rev DESC
    LIMIT 10;
    """
    top10 = pd.read_sql_query(q, conn)

top10


,game_id,date,opponent,promo_type,attendance,demand_index,ticket_rev,merch_rev,concessions_rev,parking_rev,total_rev
0,202425_HOME_04,2024-11-01,Suns,bobblehead,17644,1.472,5192554.80,311605.0,342064.75,44568.64,5890793.19
1,202425_HOME_07,2024-11-10,Warriors,bobblehead,16195,1.444,4447756.71,329108.0,335061.56,40120.03,5152046.30
2,202425_HOME_22,2024-12-29,Suns,none,17415,1.357,4420396.40,292845.0,370649.12,42823.36,5126713.88
3,202425_HOME_24,2025-01-04,Suns,giveaway,17533,1.335,4378631.66,270966.0,372607.07,41672.31,5063877.04
4,202425_HOME_30,2025-01-24,Suns,theme_night,16368,1.366,4377283.42,315395.0,295470.00,40526.19,5028674.61
5,202425_HOME_17,2024-12-14,Celtics,discount_bundle,16817,1.333,4365450.95,250971.0,312968.59,40560.50,4969951.04
6,202425_HOME_18,2024-12-16,Clippers,partner_activation,16800,1.316,4135675.61,298791.0,312191.39,39287.49,4785945.49
7,202425_HOME_39,2025-02-19,Knicks,none,16924,1.262,4122750.02,273028.0,322771.24,40103.42,4758652.68
8,202425_HOME_35,2025-02-07,Celtics,none,16650,1.302,4095424.24,276068.0,335572.02,41280.16,4748344.42
9,202425_HOME_03,2024-10-29,Nuggets,discount_bundle,16963,1.207,4015576.14,254586.0,320249.63,39730.97,4630142.74


In [10]:
df_games.to_csv("games.csv", index=False)
df_ticket_sales.to_csv("ticket_sales.csv", index=False)
df_merch_sales.to_csv("merch_sales.csv", index=False)
df_other_sales.to_csv("other_sales.csv", index=False)

print("Wrote: games.csv, ticket_sales.csv, merch_sales.csv, other_sales.csv, and SQLite DB:", DB_PATH)


Wrote: games.csv, ticket_sales.csv, merch_sales.csv, other_sales.csv, and SQLite DB: lakers_sales_kb.sqlite
